In [ ]:
from typing import List, Tuple

def obtener_cantidad_de_puntos_por_ciclo(cantidad_inicial: int, cantidad_pares_ordenados: int) -> int:
    """Obtener una cantidad de puntos por ciclos que cumpla que
        (<<cantidad_puntos_por_ciclo>> - <<cantidad_pares_ordenados>>) % (<<cantidad_pares_ordenados>> - 1) = 0

    Parameters
    ----------
    cantidad_inicial : int
        Cantidad de puntos por ciclo inicial
    cantidad_pares_ordenados : int
        Cantidad de pares ordenados

    Returns
    -------
    int
        Cantidad de puntos por ciclo final
    """
    print(
        f"Recuerde que, dado un número de pares ordenados, usted necesita cumplir que\n"
        f"\t(<<cantidad_puntos_por_ciclo>> - <<cantidad_pares_ordenados>>) % (<<cantidad_pares_ordenados>> - 1) = 0\n"
        f"En este caso, dado {cantidad_pares_ordenados} pares ordenados, la igualdad queda de la siguiente manera\n"
        f"\t({cantidad_inicial} - {cantidad_pares_ordenados}) % ({cantidad_pares_ordenados} - 1) != 0\n"
        f"\t({cantidad_inicial - cantidad_pares_ordenados}) % ({cantidad_pares_ordenados - 1}) = {(cantidad_inicial - cantidad_pares_ordenados) % (cantidad_pares_ordenados - 1)}\n"
        f"Se ajustará el número de puntos al valor próximo superior que permite cumplir esta última condición.\n"
        f"La cantidad de puntos fue actualizada de {cantidad_inicial}", end=" "
    )
    
    while (cantidad_inicial - cantidad_pares_ordenados) % (cantidad_pares_ordenados - 1) != 0:
        cantidad_inicial += 1
        
    print(
        f"a {cantidad_inicial}\n"
        f"En este caso,\n"
        f"\t({cantidad_inicial} - {cantidad_pares_ordenados}) % ({cantidad_pares_ordenados} - 1) != 0\n"
        f"\t({cantidad_inicial - cantidad_pares_ordenados}) % ({cantidad_pares_ordenados - 1}) = 0"
    )
    
    return int(cantidad_inicial)

def obtener_funcion_deseada(pares_ordenados: List[Tuple[int, int]], cantidad_puntos_por_ciclo: int, cantidad_de_ciclos: int) -> Tuple[np.ndarray, np.ndarray, int]:
    """Obtener una función deseada a partes de un conjunto de pares ordenados

    Parameters
    ----------
    pares_ordenados : List[Tuple[int, int]]
        Pares ordenados que se desean satisfacer en la función deseada
    cantidad_puntos_por_ciclo : int
        Cantidad de muestras totales que se desea de la función
    cantidad_de_ciclos : int
        Cantidad de ciclos totales que se desea de la función

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        Tupla con primer elemento con valores del eje de abscisa, segundo elemento con valores del eje de ordenadas 
        y tercer elemento cantidad de puntos en cada recta de la función
    """
    try: 
        assert (cantidad_puntos_por_ciclo - len(pares_ordenados)) % (len(pares_ordenados) - 1) == 0
    except AssertionError:
        print(f"La cantidad de puntos solicitada no es divisible en el número de rectas necesarias, es decir {len(pares_ordenados) - 1}.")
        cantidad_puntos_por_ciclo = obtener_cantidad_de_puntos_por_ciclo(cantidad_inicial=cantidad_puntos_por_ciclo, cantidad_pares_ordenados=len(pares_ordenados))
    finally:
        cantidad_puntos_por_tramo = int((cantidad_puntos_por_ciclo - len(pares_ordenados)) / (len(pares_ordenados) - 1))
    
    x_max = max(pares_ordenados, key = lambda t: t[0])[0]
    pares_ordenados_sorted = sorted(pares_ordenados, key=lambda x: x[0])

    senial_en_x = []
    senial_en_y = []
    for index in range(len(pares_ordenados_sorted)):    
        
        if index == len(pares_ordenados_sorted) - 1:
            break
        
        x_0, y_0 = pares_ordenados_sorted[index]
        x_1, y_1 = pares_ordenados_sorted[index + 1]
        
        pendiente = (y_1 - y_0) / (x_1 - x_0)
        ordenada = (y_1 * x_0 - y_0 * x_1) / (x_0 - x_1)
        
        recta = lambda x: pendiente * x + ordenada
        
        puntos_en_tramo = np.linspace(
            start=x_0,
            stop=x_1,
            num=int(cantidad_puntos_por_tramo)
        )
        
        for index_aux, punto in enumerate(puntos_en_tramo):    
            if index < len(pares_ordenados_sorted) - 2:
                if index_aux < len(puntos_en_tramo) - 1:
                    senial_en_x.append(punto)
                    senial_en_y.append(recta(punto))
            else:
                senial_en_x.append(punto)
                senial_en_y.append(recta(punto))

    senial_en_x = np.array(senial_en_x)
    senial_en_y = np.array(senial_en_y)

    senial_en_x_repetida = senial_en_x
    for ciclo in range(1, cantidad_de_ciclos):
        if ciclo < cantidad_de_ciclos - 1:
            senial_en_x_repetida = np.concatenate(
                [
                    senial_en_x_repetida,
                    senial_en_x + ciclo * x_max
                ]
            )
        else:
            senial_en_x_repetida = np.concatenate(
                [
                senial_en_x_repetida,
                senial_en_x + ciclo * x_max + senial_en_x[1] - senial_en_x[0]
                ]
            )
            
    senial_en_y_repetida = np.tile(senial_en_y, reps=cantidad_de_ciclos)
    
    return senial_en_x_repetida, senial_en_y_repetida, cantidad_puntos_por_tramo

def graficar_funcion_con_histograma(abscisa: np.ndarray, ordenada: np.ndarray, cantidad_puntos_por_tramo: int, cantidad_de_rectas: int, tamaño_bin: int) -> None:
    """Graficar una función dados los valores de x e y junto con su histograma

    Parameters
    ----------
    abscisa : np.ndarray
        Arreglo con los valores que toma el eje de las abscisas
    ordenada : np.ndarray
        Arreglo con los valores que toma el eje de las ordenadas
    cantidad_puntos_por_tramo : int
        Cantidad de puntos que contiene cada recta de la función
    cantidad_de_rectas: int
        Cantidad de rectas que contiene la función
    tamaño_bin: int
        Tamaño de bin que va a ser utilizado en el histograma
    """
    font_size = 15
    points = [
        (cantidad_puntos_por_tramo - 1) * numero_de_recta
        for numero_de_recta in range(cantidad_de_rectas + 1)
    ]
    
    _, ax = plt.subplots(1, 2, figsize=(35, 10))

    ax[0].plot(abscisa, ordenada, markevery=points, marker='o', markersize=10)
    ax[0].set_title("Señal creada a partir de pares ordenados", fontsize=font_size)
    ax[0].set_xlabel("x", fontsize=font_size)
    ax[0].set_ylabel("f(x)", fontsize=font_size)
    ax[1].hist(ordenada, bins=tamaño_bin)
    ax[1].set_title("Histograma de la señal", fontsize=font_size)
    ax[1].set_xlabel("f(x)", fontsize=font_size)
    ax[1].set_ylabel("Número de muestras", fontsize=font_size)
    plt.show()